In [97]:
import pandas as pd
import numpy as np
df1 = pd.read_csv("batch1_50rows.csv")
df2 = pd.read_csv("batch1_50rows_correct.csv")

In [98]:
columns = [['idiom_'+str(i), 'instance_'+str(i), 'label_'+str(i), 'meaning_'+str(i)] for i in range(5)]

# reshape df1
df1_melted = pd.concat([df1[cols].rename(columns=lambda x: x.split('_')[0]) for cols in columns])
df1_melted.drop_duplicates(inplace=True)

# reshape df2
df2_melted = pd.concat([df2[cols].rename(columns=lambda x: x.split('_')[0]) for cols in columns])
df2_melted.drop_duplicates(inplace=True)

In [99]:
df2_melted['label'].value_counts()

2    93
1    86
0    71
Name: label, dtype: int64

In [100]:
new_df = df2_melted[(df2_melted['label'] == 0) | (df2_melted['label'] == 1)].reset_index(drop=True)
n_to_drop = int(sum(new_df['label'] == 0) / 4)
# get the indices of the rows to drop
drop_indices = new_df[new_df['label'] == 0].sample(n=n_to_drop, random_state=42).index
# drop these rows
new_df = new_df.drop(drop_indices).reset_index(drop=True)

In [101]:
new_df['label'].value_counts()

1    86
0    54
Name: label, dtype: int64

In [102]:
# calculate the groups for each 5 rows
groups = np.arange(len(new_df)) // 5  

# set the new index and unstack
temp = new_df.set_index([groups, new_df.groupby(groups).cumcount()])
new_df_unstacked = temp.unstack().sort_index(axis=1, level=1)

# adjust the column names and reset the index
new_df_unstacked.columns = [f'{x}_{y}' for x, y in new_df_unstacked.columns]
new_df_unstacked.reset_index(drop=True, inplace=True)

# replace NaNs in label columns with -1 and convert them to int
labels = ['label_0', 'label_1', 'label_2', 'label_3', 'label_4']
for label in labels:
    new_df_unstacked[label] = new_df_unstacked[label].fillna(-1).astype(int)

# replace other NaNs with "NOT AVAILABLE: SELECT A RANDOM OPTION"
new_df_unstacked.fillna("NOT AVAILABLE: SELECT A RANDOM OPTION", inplace=True)

In [103]:
cnt = 0
for i in range(5):
    cnt += new_df_unstacked[f'label_{i}'].value_counts()
cnt

1    86
0    54
Name: label_0, dtype: int64

In [104]:
new_df_unstacked.to_csv("batch1_correction.csv", index=False)

In [26]:
merged['label'].value_counts()

1    86
Name: label, dtype: int64